In [ ]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class FacebookCommentCrawler:
    def __init__(self, chrome_driver_path, cookies_file, page_url):
        self.chrome_driver_path = chrome_driver_path
        self.cookies_file = cookies_file
        self.page_url = page_url
        self.driver = None
        self.comments_data = []
        
        # Tách phần handle của fanpage, ví dụ: "bistar.ecopark"
        self.page_handle = self.page_url.strip("/")
        # Lấy chuỗi sau "facebook.com/"
        if "facebook.com/" in self.page_handle:
            self.page_handle = self.page_handle.split("facebook.com/")[-1]
        # Nếu còn dấu "/" thì bỏ đi
        if "/" in self.page_handle:
            self.page_handle = self.page_handle.split("/")[0]
        
        # Để loại bỏ bình luận trùng lặp
        self.seen_comments = set()  # Lưu các tuple (post_url, commenter_name, comment_text, comment_date)

    def setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        
        service = Service(self.chrome_driver_path)
        self.driver = webdriver.Chrome(service=service, options=chrome_options)

    def load_cookies(self):
        self.driver.get("https://www.facebook.com/")
        time.sleep(5)
        with open(self.cookies_file, "r", encoding="utf-8") as f:
            cookies = json.load(f)
            for cookie in cookies:
                # Tránh lỗi "sameSite" không hợp lệ
                if 'sameSite' in cookie:
                    del cookie['sameSite']
                self.driver.add_cookie(cookie)
        self.driver.refresh()
        time.sleep(5)

    def get_post_links(self, max_posts=10):
        """
        Chỉ lấy link post thuộc fanpage, dựa vào page_handle và 
        chỉ những link có dạng /posts/ hoặc /permalink/.
        """
        self.driver.get(self.page_url)
        time.sleep(5)

        post_links = set()
        last_height = self.driver.execute_script("return document.body.scrollHeight")

        while len(post_links) < max_posts:
            # Tìm link dạng /posts/ hoặc /permalink/
            posts = self.driver.find_elements(By.XPATH, "//a[contains(@href, '/posts/') or contains(@href, '/permalink/')]")
            for post in posts:
                link = post.get_attribute("href")
                if not link:
                    continue
                # Bỏ query string để tránh trùng
                clean_link = link.split('?')[0]

                # Kiểm tra xem link có chứa handle fanpage không
                # (Ví dụ: /bistar.ecopark/posts/...)
                if (self.page_handle in clean_link) and ("/posts/" in clean_link or "/permalink/" in clean_link):
                    post_links.add(clean_link)
                    if len(post_links) >= max_posts:
                        break

            # Cuộn xuống để load thêm bài viết
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        print(f"Found {len(post_links)} post links (from fanpage).")
        return list(post_links)

    def expand_comments(self):
        # Click liên tục nút "Xem thêm bình luận" / "View more comments"
        while True:
            try:
                more_button = WebDriverWait(self.driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, 
                        "//div[contains(text(), 'Xem thêm bình luận') or contains(text(), 'View more comments')]"))
                )
                self.driver.execute_script("arguments[0].click();", more_button)
                time.sleep(2)
                print("Clicked 'View more comments' button.")
            except:
                # Không còn nút 'Xem thêm bình luận' nữa
                break

    def select_all_comments(self):
        """
        Chuyển chế độ hiển thị bình luận sang 'Tất cả bình luận' (nếu có).
        Vì mặc định Facebook hay để 'Phù hợp nhất' / 'Most relevant'.
        """
        try:
            # Tìm nút filter hiển thị bình luận
            filter_button = WebDriverWait(self.driver, 2).until(
                EC.element_to_be_clickable((By.XPATH,
                    "//span[contains(text(),'Phù hợp nhất') or contains(text(),'Most relevant') or contains(text(),'Bình luận hàng đầu')]"))
            )
            self.driver.execute_script("arguments[0].click();", filter_button)
            time.sleep(1)

            # Tìm và click nút 'Tất cả bình luận' / 'All comments'
            all_comments_button = WebDriverWait(self.driver, 2).until(
                EC.element_to_be_clickable((By.XPATH,
                    "//span[contains(text(),'Tất cả bình luận') or contains(text(),'All comments')]"))
            )
            self.driver.execute_script("arguments[0].click();", all_comments_button)
            time.sleep(1)
            print("Switched to 'All comments' mode.")
        except:
            print("Could not switch to 'All comments' mode (maybe already on).")

    def get_comments_from_post(self, post_url):
        self.driver.get(post_url)
        time.sleep(5)

        # Chuyển sang chế độ hiển thị 'Tất cả bình luận'
        self.select_all_comments()
        time.sleep(2)

        # Mở rộng tất cả bình luận
        self.expand_comments()
        time.sleep(2)

        print(f"Scraping comments from: {post_url}")

        # Lấy nội dung bài đăng
        try:
            post_content_elem = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@role='article']//div[@dir='auto']"))
            )
            post_content = post_content_elem.text.strip()
        except:
            post_content = "Không thể trích xuất nội dung bài đăng"

        # Tìm các container bình luận
        comment_elements = self.driver.find_elements(By.XPATH, 
            "//div[@aria-label and (contains(@aria-label, 'Bình luận') or contains(@aria-label, 'comment'))]"
        )

        print(f"Found {len(comment_elements)} comment containers.")

        for comment in comment_elements:
            try:
                # Tên người bình luận và link
                try:
                    commenter_elem = comment.find_element(By.XPATH, ".//a[contains(@href, 'facebook.com') and @role='link']")
                    commenter_name = commenter_elem.text.strip()
                    username = commenter_elem.get_attribute("href").split('?')[0]
                except:
                    commenter_name = "Ẩn danh"
                    username = "Không xác định"

                # Nội dung bình luận
                try:
                    comment_text_elem = comment.find_element(By.XPATH, ".//div[@dir='auto']")
                    comment_text = comment_text_elem.text.strip()
                except:
                    comment_text = "Không thể trích xuất nội dung"

                # Thời gian bình luận
                # Facebook thường để trong <abbr title="..."> hoặc hiển thị text "xx tuần trước"
                try:
                    comment_date_elem = comment.find_element(By.XPATH, ".//a//abbr")
                    
                    comment_date = comment_date_elem.text.strip()
                    if not comment_date:
                        # Nếu rỗng, thử lấy title
                        comment_date = comment_date_elem.get_attribute("title")
                except:
                    comment_date = "Không xác định"

                # Số reaction của bình luận
                try:
                    reaction_elem = comment.find_element(By.XPATH, 
                        ".//span[contains(@aria-label, 'lượt thích') or contains(@aria-label, 'reactions')]")
                    comment_reactions = reaction_elem.text.strip()
                except:
                    comment_reactions = "0"

                # Kiểm tra trùng lặp comment
                comment_key = (post_url, commenter_name, comment_text, comment_date)
                if comment_key in self.seen_comments:
                    # Đã tồn tại => bỏ qua
                    continue
                else:
                    self.seen_comments.add(comment_key)

                # Lưu vào danh sách kết quả
                self.comments_data.append({
                    'post_url': post_url,
                    'post_content': post_content,
                    'username': username,
                    'commenter_name': commenter_name,
                    'comment_date': comment_date,
                    'comment_text': comment_text,
                    'comment_reactions': comment_reactions
                })
                print(f"Fetched comment: {comment_text[:30]}...")
            except Exception as ex:
                print("Error processing a comment:", ex)
                continue

    def get_reactions_and_shares(self, post_url):
        """
        Lấy lượt reaction và lượt share của bài đăng.
        """
        self.driver.get(post_url)
        time.sleep(5)

        # Lấy thông tin reaction
        try:
            reactions_elem = WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.XPATH, 
                    "//div[contains(@aria-label, 'lượt phản ứng') or contains(@aria-label, 'reactions')]"))
            )
            reactions = reactions_elem.get_attribute("aria-label") if reactions_elem else "0"
        except:
            reactions = "0"

        # Lấy thông tin share
        try:
            shares_elem = self.driver.find_element(By.XPATH, 
                "//span[contains(text(), 'chia sẻ') or contains(text(), 'shares')]")
            shares = shares_elem.text.strip() if shares_elem else "0"
        except:
            shares = "0"

        print(f"Post reactions: {reactions}, Shares: {shares}")
        return reactions, shares

    def save_to_excel(self, filename="facebook_comments.xlsx"):
        df = pd.DataFrame(self.comments_data)
        df.to_excel(filename, index=False)
        print(f"Data saved to {filename}")

    def crawl_fanpage(self, max_posts=10):
        try:
            self.setup_driver()
            self.load_cookies()
            post_urls = self.get_post_links(max_posts=max_posts)
            for post_url in post_urls:
                self.get_comments_from_post(post_url)
                reactions, shares = self.get_reactions_and_shares(post_url)
                # Gán thông tin phản ứng của bài đăng cho các bình luận của bài đó
                for comment in self.comments_data:
                    if comment['post_url'] == post_url:
                        comment['post_reactions'] = reactions
                        comment['post_shares'] = shares
                time.sleep(2)
            self.save_to_excel()
        finally:
            self.driver.quit()


if __name__ == "__main__":
    # Thay đổi các biến bên dưới cho phù hợp
    chrome_driver_path = "C:/Users/THINK/Desktop/NLP Vnua/Selenium/chromedriver-win64/chromedriver.exe"
    cookies_file = "cookies.json"
    page_url = "https://www.facebook.com/bistar.ecopark"

    crawler = FacebookCommentCrawler(chrome_driver_path, cookies_file, page_url)
    crawler.crawl_fanpage(max_posts=10)


Found 10 post links (from fanpage).
Could not switch to 'All comments' mode (maybe already on).
Scraping comments from: https://www.facebook.com/bistar.ecopark/posts/pfbid0xeoJeqiweMDtMZVjA3FGs5J1y8vqJMT6c5m4LZdoVTXcKfDgQnMgudF5t5PaNiwol
Found 5 comment containers.
Fetched comment: Hóng anh chai reaction lại cái...
Fetched comment: ...
Fetched comment: Không thể trích xuất nội dung...
Post reactions: 0, Shares: 146 lượt chia sẻ
Could not switch to 'All comments' mode (maybe already on).
Scraping comments from: https://www.facebook.com/bistar.ecopark/posts/pfbid01T5wXzX6oCdDAE8DU3CNuG1NkTjXP13Gi52W9ZGxbRh5MRzNu1cxQwG1inonSgJfl
Found 4 comment containers.
Fetched comment: Hóng anh chai reaction lại cái...
Fetched comment: Không thể trích xuất nội dung...
Post reactions: 0, Shares: 150 lượt chia sẻ
Switched to 'All comments' mode.
Scraping comments from: https://www.facebook.com/bistar.ecopark/posts/pfbid09gzN86syBLrrULGo4KXJr3f4yWBWNpcASojYQbK1vNfYRCcVGsEYnqJP6aj2XbMHl
Found 5 comment co